## Chain of Thought

In [1]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [7]:
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(model=model, temperature=0)

In [8]:
example_problem = "What is 678 * 789?"

# CoT Prompt 정의 
example_cot = '''\
Let me break this down into steps. First I'll break down 789 into hundreds, tens, and ones:

789 -> 700 + 80 + 9

Next I'll multiply 678 by each of these values, storing the intermediate results:

678 * 700 -> 678 * 7 * 100 -> 4746 * 100 -> 474600

My first intermediate result is 474600.

678 * 80 -> 678 * 8 * 10 -> 5424 * 10 -> 54240

My second intermediate result is 54240.

678 * 9 -> 6102

My third intermediate result is 6102.

My three intermediate results are 474600, 54240, and 6102.

Adding the first two intermediate results I get 474600 + 54240 -> 528840.

Adding 528840 to the last intermediate result I get 528840 + 6102 -> 534942

The final result is 534942.
'''

In [9]:
multiplication_template = ChatPromptTemplate.from_messages([
    ('human', example_problem),
    ('ai', example_cot),
    ('human', '{long_multiplication_prompt}')
])

In [10]:
multiplication_chain = multiplication_template | llm | StrOutputParser()

In [11]:
print(multiplication_chain.invoke('What is 345 * 888?'))

To calculate 345 * 888, I'll break down 888 into hundreds, tens, and ones:

888 -> 800 + 80 + 8

Next, I'll multiply 345 by each of these values, storing the intermediate results:

345 * 800 -> 345 * 8 * 100 -> 2760 * 100 -> 276000

My first intermediate result is 276000.

345 * 80 -> 345 * 8 * 10 -> 2760 * 10 -> 27600

My second intermediate result is 27600.

345 * 8 -> 2760

My third intermediate result is 2760.

My three intermediate results are 276000, 27600, and 2760.

Adding the first two intermediate results I get 276000 + 27600 -> 303600.

Adding 303600 to the last intermediate result I get 303600 + 2760 -> 306360

The final result is 306360.


## Zero-shot Chain-of-Thought Prompting

In [12]:
zero_shot_cot_prompt = ChatPromptTemplate([
    ("human", "{long_multiplication_prompt} Let's think step by step.")
])

In [13]:
zero_shot_multiplication_chain = zero_shot_cot_prompt | llm | StrOutputParser()

In [14]:
print(zero_shot_multiplication_chain.invoke('What is 345 * 888?'))

To calculate 345 * 888, let's break it down step by step.

First, we can multiply 300 by 888:

300 * 888 = 266,400

Next, we can multiply 40 by 888:

40 * 888 = 35,520

Then, we can multiply 5 by 888:

5 * 888 = 4,440

Now, let's add up the results of each multiplication:

266,400 + 35,520 + 4,440 = 306,360

Therefore, 345 * 888 = 306,360.


In [15]:
345*888

306360

## Chatbots 

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(model=model, temperature=0)

# placeholder에 전달된 메세지들은 무조건 리스트로  
# placeholder는 다수의 메세지를 한 번에 삽입할 수 있도록 해주는 키워드 
template_with_placeholder = ChatPromptTemplate.from_messages([
    ('placeholder', "{messages}"),
    ("human", "{prompt}")
])

In [18]:
messages = [
    ('human', 'The sun came up today.'),
    ('ai', 'That is wonderful!'),
    ('human', 'The sun went down today.'),
    ('ai', 'That is also wonderful!.')
]

prompt = "What happened today?"

In [20]:
template_with_placeholder.invoke({"messages": messages, "prompt": prompt})

ChatPromptValue(messages=[HumanMessage(content='The sun came up today.', additional_kwargs={}, response_metadata={}), AIMessage(content='That is wonderful!', additional_kwargs={}, response_metadata={}), HumanMessage(content='The sun went down today.', additional_kwargs={}, response_metadata={}), AIMessage(content='That is also wonderful!.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What happened today?', additional_kwargs={}, response_metadata={})])

In [21]:
chain = template_with_placeholder | llm | StrOutputParser()

chain.invoke({'messages': messages, "prompt": prompt})

"It seems like you're describing a typical day! The sun rising and setting is a natural occurrence that happens every day. It's a reminder of the Earth's rotation and the passage of time. Did anything else notable happen today, or was it a quiet day?"

In [22]:
chat_conversation_template = ChatPromptTemplate.from_messages([
    ('placeholder', '{chat_conversation}')
])

chat_chain = chat_conversation_template | llm | StrOutputParser()

chat_conversation = []

In [23]:
chat_conversation.append(('user', 'Hello, my name is Michael.'))

In [24]:
chat_chain.invoke({'chat_conversation': chat_conversation})

"Hello Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [25]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(("ai", response))

In [26]:
chat_conversation

[('user', 'Hello, my name is Michael.'),
 ('ai',
  "Hello Michael! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?")]

## Chatbots Class

In [ ]:
class Chatbot:
    def __init__(self, llm):
        # 대화 기록을 저장할 리스트 
        self.chat_conversation = []
        
        # chain을 구성하는 러너블
        parser = StrOutputParser()
        template = ChatPromptTemplate.from_messages([
            ("placeholder","{chat_conversation}")
        ])
        
        # 채팅 기록을 바탕으로 응답하는 체인 
        self.chat_chain = template | llm | parser 
        
    def chat(self, prompt):
        self.chat_conversation.append(('user',prompt))
        # 갱신된 대화 기록으로 응답 생성
        
        response = self.chat_chain.invoke({"chat_conversation":self.chat_conversation})
        self.chat_conversation.append(('ai', response))
        
        return response 
    
    def clear(self):
        self.chat_conversation = []
        
chatbot = Chatbot(llm)

In [28]:
print(chatbot.chat('Hi, my name is Michael.'))

Hi Michael! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [29]:
print(chatbot.chat('I just want to be reminded of my name please.'))

Your name is Michael.


In [31]:
print(chatbot.chat("That's really cool! Give me another."))

I think I see what's happening here! I'll give you a different one this time. Your name is... Michael. (Just kidding!) Seriously, I can give you a random name, though. How about "Astrid"?


## Chatbot With Role `system_message`

In [32]:
brief_chatbot_system_message = "You always answer as briefly and concisely as possible."

curious_chatbot_system_message = """\
You are incredibly curious, and often respond with reflections and followup questions that lean the conversation in the direction of playfully \
understanding more about the subject matters of the conversation."""

increased_vocabulary_system_message = """\
You always respond using challenging and often under-utilized vocabulary words, even when your response could be made more simply."""

In [38]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('system', system_message),
            ('placeholder', '{chat_conversation}'),
            
        ])
        
        self.system_message = system_message

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))
        
        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

In [41]:
brief_chatbot = ChatbotWithRole(llm, brief_chatbot_system_message)
curious_chatbot = ChatbotWithRole(llm, curious_chatbot_system_message)

In [47]:
print(brief_chatbot.chat('Hi, my name is hyunyoung.'))

Hyunyoung.


In [48]:
print(brief_chatbot.chat('I just want to be reminded of my name please.'))

Hyunyoung.


In [49]:
print(curious_chatbot.chat('I just want to be reminded of my name please.'))

I'm not actually aware of your name, since this is the start of our conversation. But I'm curious, how do you feel about not knowing your name in this moment? Does it feel strange or no big deal?


## Structured Output 

In [50]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for: \
- The name of the city \
- The country the city is located in.'''

prompt

'Make a JSON object representing the city Santa Clara. It should have fields for: - The name of the city - The country the city is located in.'

In [51]:
print(llm.invoke(prompt).content)

Here is a JSON object representing the city Santa Clara:

```
{
  "name": "Santa Clara",
  "country": "Cuba"
}
```

However, it's worth noting that there are multiple cities named Santa Clara around the world. If you're referring to a different Santa Clara, please let me know and I can update the country field accordingly.

For example, there's also a Santa Clara in California, USA. Here's an updated JSON object:

```
{
  "name": "Santa Clara",
  "country": "United States"
}
```

Or, if you're referring to a Santa Clara in another country, please let me know and I can update the country field.


In [56]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text.'''

In [58]:
print(llm.invoke(prompt).content)

```json
{
  "name": "Santa Clara",
  "country": "Cuba"
}
```


In [63]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.'''

llm.invoke(prompt).content

'{"name": "Santa Clara", "country": "United States"}'

In [ ]:
# json 형태의 데이터 생성 
json_city = llm.invoke(prompt).content
json_city

'{"name": "Santa Clara", "country": "United States"}'

In [65]:
import json 

python_city = json.loads(json_city)

for k, v in python_city.items():
    print(f"{k}: {v}")

name: Santa Clara
country: United States


In [66]:
json_city_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the city {city_name}. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

parser = StrOutputParser()
chain = json_city_template | llm | parser 

print(chain.invoke({'city_name': 'Santa Clara'}))

{"name": "Santa Clara", "country": "United States"}


### Simple JSON Parsing

In [ ]:
from langchain_core.output_parsers import SimpleJsonOutputParser
from langchain_core.runnables import RunnableLambda

# parse_to_dict = RunnableLambda(lambda response: json.loads(response.content))
json_parser = SimpleJsonOutputParser()

sci_fi_books = [
    {"book_title": "Dune"},
    {"book_title": "Neuromancer"},
    {"book_title": "Snow Crash"},
    {"book_title": "The Left Hand of Darkness"},
    {"book_title": "Foundation"}
]

book_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the following book {book_title}. \
It should have fields for:
- The name of the book
- The author of the book
- The year the book is published.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

chain = book_template | llm | json_parser 

result = chain.batch(sci_fi_books)

In [70]:
result[0]['name']

'Dune'

## Structured Output with Pydantic 

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field # 자료형을 강제하기 위한 라이브러리 

In [72]:
book_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the details of the following book: {book_title}. \
It should have fields for:
- The title of the book.
- The author of the book.
- The year the book was originally published.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

### Structured Data as a Class

In [76]:
class Book:
    """Information about a book."""
    
    def __init__(self, title, author, year_of_publication):
        self.title: str = title
        self.author: str = author
        self.year_of_publication: int = year_of_publication

In [77]:
class Book(BaseModel):
    title: str = Field(...,description = "The title of the book")
    author: str = Field(..., description="The author of the book")
    year_of_publication: str = Field(description="The year the book was published")

In [ ]:
# Book class에 대한 instructions을 만들어 줌 
parser = JsonOutputParser(pydantic_object=Book)
format_instructions = parser.get_format_instructions() # 구조 설명 
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"description": "The title of the book", "title": "Title", "type": "string"}, "author": {"description": "The author of the book", "title": "Author", "type": "string"}, "year_of_publication": {"description": "The year the book was published", "title": "Year Of Publication", "type": "string"}}, "required": ["title", "author", "year_of_publication"]}
```


### Using Formatting Instructions in Prompts 

In [80]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        "Generate JSON about the user input according to the provided format instructions.\n" +
        "Input: {input}\n" +
        "Format instructions {format_instructions}")
    )
])

In [81]:
chain = template | llm | parser #parser = JsonOutputParser(pydantic_object=Book)

In [ ]:
chain.invoke({
    "input": "East of Eden",
    "format_instructions": format_instructions # 원하는 구조의 instructions을 전달 
})

{'title': 'East of Eden',
 'author': 'John Steinbeck',
 'year_of_publication': '1952'}

## Exercise: Leverage Pydantic for Structured Data Generation

In [83]:
city_names = ['Tokyo', 'Busan', 'Cairo', 'Perth']

In [ ]:
class City(BaseModel):
    name: str = Field(..., description = "The name of the city")
    country: str = Field(..., description="The country that the city is located within")
    capital: bool = Field(..., description=" Whether or not the city is the capital city of the country it is located in.")
    population: float = Field(..., description="The population of the city.")

### LLM 에게 response structure 전달 
parser = JsonOutputParser(pydantic_object=City)

format_instructions = parser.get_format_instructions()

template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        """Generate JSON about the user input according to the provided format instructions.
            Format instructions {format_instructions}""")
    )
])

chain = template | llm | parser
chain.batch(city_names)

[{'city': 'Tokyo',
  'country': 'Japan',
  'population': 38200000,
  'language': 'Japanese',
  'currency': 'Japanese Yen'},
 {'name': 'Busan',
  'type': 'city',
  'location': {'country': 'South Korea', 'region': 'Southeastern Korea'},
  'population': 3430000,
  'description': 'A major port city and the second-largest city in South Korea.'},
 {'user': {'name': 'John Doe', 'age': 30, 'city': 'Cairo'}},
 {'city': 'Perth',
  'country': 'Australia',
  'population': 2150000,
  'latitude': -31.9523,
  'longitude': 115.8586}]